In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [37]:
# Read, then decode for py2 compat.
#text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
filename = "discurso.txt"
text = open(filename, encoding="utf-8").read()
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')


Length of text: 25328 characters


In [38]:
# Take a look at the first 250 characters in text
print(text[:250])

estamos celebrando este evento de nuevo en nuestra sede corporativa de boadilla para minimizar los riesgos sanitarios
que podría conllevar el desplazamiento de nuestros equipos y accionistas a
cantabria deseamos volver pronto a santander
y me gustarí


In [39]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

51 unique characters


In [40]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)


In [41]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [42]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [43]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(25328,), dtype=int64, numpy=array([24, 38, 39, ..., 28, 20, 38], dtype=int64)>

In [44]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [45]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

e
s
t
a
m
o
s
 
c
e


In [46]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [47]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'e' b's' b't' b'a' b'm' b'o' b's' b' ' b'c' b'e' b'l' b'e' b'b' b'r'
 b'a' b'n' b'd' b'o' b' ' b'e' b's' b't' b'e' b' ' b'e' b'v' b'e' b'n'
 b't' b'o' b' ' b'd' b'e' b' ' b'n' b'u' b'e' b'v' b'o' b' ' b'e' b'n'
 b' ' b'n' b'u' b'e' b's' b't' b'r' b'a' b' ' b's' b'e' b'd' b'e' b' '
 b'c' b'o' b'r' b'p' b'o' b'r' b'a' b't' b'i' b'v' b'a' b' ' b'd' b'e'
 b' ' b'b' b'o' b'a' b'd' b'i' b'l' b'l' b'a' b' ' b'p' b'a' b'r' b'a'
 b' ' b'm' b'i' b'n' b'i' b'm' b'i' b'z' b'a' b'r' b' ' b'l' b'o' b's'
 b' ' b'r' b'i'], shape=(101,), dtype=string)


In [48]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'estamos celebrando este evento de nuevo en nuestra sede corporativa de boadilla para minimizar los ri'
b'esgos sanitarios\nque podr\xc3\xada conllevar el desplazamiento de nuestros equipos y accionistas a\ncantabria'
b' deseamos volver pronto a santander\ny me gustar\xc3\xada comenzar enviando un mensaje de apoyo al pueblo ucr'
b'aniano que est\xc3\xa1 sufriendo esta terrible cat\xc3\xa1strofe\nhumanitaria a todos los que han perdido familiares'
b' y amigos y a los que han\ntenido que abandonar sus hogares ahora es el momento de apoyar como en los '


In [49]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [50]:
dataset = sequences.map(split_input_target)

In [51]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'estamos celebrando este evento de nuevo en nuestra sede corporativa de boadilla para minimizar los r'
Target: b'stamos celebrando este evento de nuevo en nuestra sede corporativa de boadilla para minimizar los ri'


In [52]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [79]:
for input_example, target_example in dataset.take(1):
    print("Input :", input_example.numpy().shape)
    print("Target:", target_example.numpy().shape)

Input : (64, 100)
Target: (64, 100)


In [53]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [54]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [81]:
print(len(ids_from_chars.get_vocabulary()))
print(embedding_dim)
print(rnn_units)

52
256
1024


In [55]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [56]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 52) # (batch_size, sequence_length, vocab_size)


In [82]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
print(sampled_indices)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

tf.Tensor(
[[24]
 [ 6]
 [33]
 [12]
 [50]
 [50]
 [25]
 [ 1]
 [31]
 [ 9]
 [51]
 [19]
 [25]
 [ 0]
 [17]
 [ 7]
 [39]
 [28]
 [31]
 [ 7]
 [ 8]
 [ 5]
 [50]
 [ 0]
 [24]
 [46]
 [28]
 [10]
 [49]
 [39]
 [45]
 [50]
 [48]
 [20]
 [46]
 [ 1]
 [ 6]
 [17]
 [38]
 [28]
 [ 7]
 [11]
 [ 9]
 [22]
 [ 1]
 [43]
 [28]
 [40]
 [28]
 [20]
 [18]
 [49]
 [17]
 [50]
 [36]
 [ 7]
 [41]
 [13]
 [ 2]
 [50]
 [11]
 [ 1]
 [37]
 [ 9]
 [15]
 [28]
 [16]
 [12]
 [51]
 [44]
 [48]
 [18]
 [28]
 [ 2]
 [20]
 [30]
 [22]
 [11]
 [11]
 [27]
 [26]
 [36]
 [46]
 [51]
 [17]
 [23]
 [35]
 [50]
 [19]
 [13]
 [ 4]
 [22]
 [35]
 [17]
 [17]
 [39]
 [32]
 [15]
 [20]
 [30]], shape=(100, 1), dtype=int64)


In [58]:
sampled_indices

array([40,  5, 17, 48,  6, 23, 50, 16,  0, 40,  0, 45,  7, 47,  3, 27, 21,
        7,  9, 35, 31, 33, 43, 30, 39, 12, 29,  1, 32, 30, 23, 43, 32, 25,
       48, 48, 40, 23, 15, 30, 48,  2, 28, 29, 20, 27,  9, 28,  1, 28, 16,
        0, 33, 22, 11,  6, 43,  3, 38, 23, 22, 15, 38, 41, 18, 23, 29,  1,
       41, 29, 15, 23, 48,  1, 28, 51,  0, 14, 20, 45, 42, 28, 12,  6,  5,
       47, 12, 31, 43, 19, 32, 11, 45, 38, 21, 40, 27, 25, 29, 23],
      dtype=int64)

In [59]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'etornos muy atractivos\nya esperamos que en este a\xc3\xb1o en el a\xc3\xb1o 22 pago nec sea rentable el d\xc3\xada 7 el\nc'

Next Char Predictions:
 b'u-A\xc3\xad.d\xc3\xb39[UNK]u[UNK]z0\xc3\xa9%hb02plnxkt5j\nmkdxmf\xc3\xad\xc3\xadud8k\xc3\xad ijah2i\ni9[UNK]nc4.x%sdc8sv[dj\nvj8d\xc3\xad\ni\xc3\xba[UNK]7azwi5.-\xc3\xa95lx]m4zsbuhfjd'


In [60]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [61]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 52)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.9522295, shape=(), dtype=float32)


In [62]:
tf.exp(example_batch_mean_loss).numpy()

52.051285

In [63]:
model.compile(optimizer='adam', loss=loss)

In [64]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [71]:
EPOCHS = 60
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
3/3 [==============================] - 11s 4s/step - loss: 2.7627
Epoch 2/30
3/3 [==============================] - 10s 4s/step - loss: 2.7127
Epoch 3/30
3/3 [==============================] - 10s 4s/step - loss: 2.6597
Epoch 4/30
3/3 [==============================] - 10s 4s/step - loss: 2.6027
Epoch 5/30
3/3 [==============================] - 10s 4s/step - loss: 2.5562
Epoch 6/30
3/3 [==============================] - 10s 4s/step - loss: 2.4993
Epoch 7/30
3/3 [==============================] - 11s 4s/step - loss: 2.4542
Epoch 8/30
3/3 [==============================] - 11s 4s/step - loss: 2.4126
Epoch 9/30
3/3 [==============================] - 11s 4s/step - loss: 2.3903
Epoch 10/30
3/3 [==============================] - 11s 4s/step - loss: 2.3602
Epoch 11/30
3/3 [==============================] - 12s 4s/step - loss: 2.3311
Epoch 12/30
3/3 [==============================] - 11s 4s/step - loss: 2.3076
Epoch 13/30
3/3 [==============================] - 11s 4s/step - loss: 2.

In [66]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [72]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [73]:
start = time.time()
states = None
next_char = tf.constant(['dividendo'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

dividendo cejio res acta es pompradilóz al prunto deftiondes
ros lubloxr ecrorcos cémos a astel conciciy eusorodo de
nuentrapabili do er gesdoa det vol degtos unovesestano ecicitados damobras acioso cimnodos de
benternimpera y prerar acr y a2
míy los 3 cosara a teraños lastor furevide en perotarita
poradempiartaratia
se cenoner sato pra cicien des dementios os suerero siciaños un cos
clon cle al
uande lo cajompoy al que afrere laszazada a5
701% nuelos resitimbiblera nes parta a la pomémocian an al tuy lativar snelcio de
que lasinto al menos lemecmos y ajertra mos mensiano de ciestrenos el
ruy
sus pero iccianmicla au testro bajosol an l pumamo resza
momesera an empietor paciparueste sa digibamha la ensimo necla escicicimay de aújpapas
in momávaciticio e 202 lertiisdes pa ciemilomás
de en as recrome mos pasto unsodo accénta ladofen
nefimimúnte mitos1dá do men a
melra secorarmos rusa demos do ea coral y 1f25
25 pre entefociedor o
por nemencocabse calo selís a estosas do
as centartros serc